<a href="https://colab.research.google.com/github/MocT117/Another-one-/blob/master/sem_foro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

st.set_page_config(page_title="Semáforo Fechas", page_icon=":bar_chart:", layout="centered")

st.title("Dashboard Semáforo Fechas")
st.write("Carga tu archivo Excel para visualizar el semáforo de fechas:")

archivo = st.file_uploader("Sube tu archivo Excel", type=["xlsx"])

if archivo:
    df = pd.read_excel(archivo)

    # Convertir a fecha
    df['Fecha de PO'] = pd.to_datetime(df['Fecha de PO'], errors='coerce')
    df['EXW Real'] = pd.to_datetime(df['EXW Real'], errors='coerce')

    # Filtrar solo filas con EXW Real
    df_validas = df[df['EXW Real'].notna()].copy()

    # Calcular diferencia en días
    df_validas['Diferencia días calculada'] = (df_validas['EXW Real'] - df_validas['Fecha de PO']).dt.days

    # Semáforo
    def status_semáforo(dias):
        if pd.isna(dias):
            return 'Sin fecha'
        elif 0 <= dias <= 6:
            return 'En tiempo'
        elif dias >= 7:
            return 'On Delay'
        else:
            return 'Revisar'
    df_validas['Semáforo'] = df_validas['Diferencia días calculada'].apply(status_semáforo)

    # Mostrar tabla
    st.subheader("Tabla con Semáforo (solo registros con EXW Real)")
    st.dataframe(df_validas[['Fecha de PO', 'EXW Real', 'Diferencia días calculada', 'Semáforo']])

    # Gauge chart: mostrar promedio de la diferencia de días
    st.subheader("Diferencia Fecha PO vs EXW Real")

    # Calcula el promedio y el número de registros considerados
    if not df_validas['Diferencia días calculada'].dropna().empty:
        promedio_dias = df_validas['Diferencia días calculada'].mean()
    else:
        promedio_dias = 0
    num_filas = len(df_validas)

    # Definir límites para el gauge
    minimo = 0
    maximo = max(18, int((df_validas['Diferencia días calculada'].max() // 6 + 1) * 6))  # múltiplos de 6, mínimo 18

    # Ticks en múltiplos de 6
    tick_vals = list(range(minimo, maximo+1, 6))

    fig = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = promedio_dias,
        number = {'suffix': " días", 'font': {'size': 36}},
        title = {'text': "Diferencia Fecha PO vs EXW Real"},
        gauge = {
            'axis': {'range': [minimo, maximo], 'tickmode': 'array', 'tickvals': tick_vals, 'tickfont': {'size': 16}},
            'steps': [
                {'range': [minimo, 6], 'color': "#ABEBC6"},
                {'range': [6, 14], 'color': "#F9E79F"},
                {'range': [14, maximo], 'color': "#F5B7B1"}
            ],
            'bar': {'color': "black", 'thickness': 0.25},  # línea más delgada
        }
    ))

    st.plotly_chart(fig)
    # Mostrar número de registros justo debajo del gauge, centrado y con estilo
    st.markdown(
        f"<div style='text-align:center; font-size:18px; margin-top:-30px; margin-bottom:20px;'><b>Nº de registros considerados: {num_filas}</b></div>",
        unsafe_allow_html=True
    )

    st.info("Dashboard generado por isra5 🤖")

else:
    st.warning("Sube un archivo para ver el dashboard.")



